In [1]:
import Data.Maybe
import qualified Data.IntPSQ as H
import qualified Data.HashMap as M
import qualified Data.Char as C
import Control.Monad.State.Lazy

In [2]:
type Vertex = Int
type Score = Int
type Heap = H.IntPSQ Score Vertex
type Map = M.Map Vertex Score

In [3]:
rawFileString <- readFile "./data/dijkstraData.txt"

In [4]:
adjList = M.fromList . map (splitVE . words) . lines $ rawFileString
    where 
        splitVE (x:xs) = (read x :: Int, parseEW xs)
        parseEW = map (\x -> read $ "("++x++")" :: (Int, Int))

In [5]:
insert :: Vertex -> Score -> Heap -> Heap
insert v s = H.insert v s 0

deleteView :: Vertex -> Heap -> (Vertex, Score, Heap)
deleteView v h = 
    case H.deleteView v h of
        Just (score, value, h_del) -> (v, score, h_del)
        Nothing -> (v, 1000000, h)
        
minView :: Heap -> (Vertex, Score, Heap)
minView h = 
    case H.minView h of
        Just (vertex, score, value, h_pop) -> (vertex, score, h_pop)
        Nothing -> (-1, -1, H.empty)

In [27]:
getFrontier :: [Vertex] -> Vertex -> [(Vertex, Score)]
getFrontier vss v = filter (\x -> (fst x `notElem` vss)) . getCandi $ v

getCandi :: Vertex -> [(Vertex, Score)]
getCandi v = 
    case M.lookup v adjList of
        Just candiEs -> candiEs
        Nothing -> []

updateWEachV :: Vertex -> Score -> Heap -> Heap
updateWEachV v s h = let
    (_, prev_s, h_del) = deleteView v h
    in
    if s < prev_s
    then insert v s h_del
    else insert v prev_s h_del

update :: [Vertex] -> Heap -> Vertex -> Score -> Heap
update vss h0 v0 s0 = foldl (\h -> \(v_neigh, s_path) -> updateWEachV v_neigh s_path h) h0 $ map (\(v, s) -> (v, s+s0)) $ getFrontier vss v0

dijkstra :: Map -> State Heap Map
dijkstra m = do
    h <- get
    let (v_pop, s_pop, h_pop) = minView h
    let m_ran = M.insert v_pop s_pop m
    let coveredV = M.keys m_ran 
    let h_after = update coveredV h_pop v_pop s_pop 
    put h_after
    return m_ran

Line 2: Eta reduce
Found:
getFrontier vss v
  = filter (\ x -> (fst x `notElem` vss)) . getCandi $ v
Why not:
getFrontier vss = filter (\ x -> (fst x `notElem` vss)) . getCandiLine 2: Redundant bracket
Found:
\ x -> (fst x `notElem` vss)
Why not:
\ x -> fst x `notElem` vssLine 6: Replace case with fromMaybe
Found:
case M.lookup v adjList of
    Just candiEs -> candiEs
    Nothing -> []
Why not:
Data.Maybe.fromMaybe [] (M.lookup v adjList)Line 19: Collapse lambdas
Found:
\ h -> \ (v_neigh, s_path) -> updateWEachV v_neigh s_path h
Why not:
\ h (v_neigh, s_path) -> updateWEachV v_neigh s_path h

In [31]:
runDijkstra :: Map -> Heap -> Map
runDijkstra m h = let
    (m', h') = runState (dijkstra m) h
    in
    if (length . M.keys $ m') == (length . M.keys $ m)
    then m
    else runDijkstra m' h'

In [32]:
sPMap = runDijkstra initMap initHeap

In [33]:
genSolution :: M.Map Vertex Score -> [Vertex] -> [Score]
genSolution m vs = map (\v -> parse . M.lookup v $ m) vs
    where
        parse Nothing = 1000000
        parse (Just x) = x

Line 2: Eta reduce
Found:
genSolution m vs = map (\ v -> parse . M.lookup v $ m) vs
Why not:
genSolution m = map (\ v -> parse . M.lookup v $ m)

In [34]:
genSolution sPMap [7,37,59,82,99,115,133,165,188,197]

[2599,2610,2947,2052,2367,2399,2029,2442,2505,3068]